In [1]:
from pyspark import SparkContext

Un RDD (Resilient Distributed Dataset) es la estructura básica de datos en Spark.
Es una colección de datos distribuida en varios nodos de un cluster y permite realizar operaciones en paralelo.

In [2]:
# Crea un contexto de Spark , para hacer la conexion entre mi codigo y el motor de Spark
#master = 'local' -> significa que spark se ejecutara en la maquina local usando solo nucleo de CPU 'local[*]' para usar todos los nucleos disponibles ,, En un entorno distribuido, podrías usar algo como 'yarn' o 'spark://<host>:<port>'.
# El parámetro appName define el nombre de la aplicación en Spark, útil para monitorear el trabajo en la interfaz web de Spark UI.
sc = SparkContext(master = 'local',appName =  'TransformacionesAcciones')

In [ ]:
data = [1,2,3,4]
rdd = sc.parallelize(data)

In [ ]:
type(rdd)

pyspark.rdd.RDD

In [ ]:
rdd.collect()

[1, 2, 3, 4]

In [ ]:
sc

<SparkContext master=local appName=TransformacionesAcciones>

In [3]:
path = '/content/drive/MyDrive/Spark/files/'

In [4]:
equiposOlimpicosRDD = sc.textFile(path + 'paises.csv').map(lambda line : line.split(","))

In [5]:
equiposOlimpicosRDD.take(10)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR']]

In [6]:
 # ver cuantos paises participaron
equiposOlimpicosRDD.map(lambda x : (x[2])).distinct().count()# x[2] el elemnto 2 de cada lista


231

In [ ]:
#Cuenta los rdd dentro de los primeros 20 milisegundos
equiposOlimpicosRDD.countApprox(20)

1185

In [7]:
#Importamos los csv deportista y deportista2
deportistaRDD = sc.textFile(path + 'deportista.csv').map(lambda line : line.split(","))
deportista2RDD = sc.textFile(path + 'deportista2.csv').map(lambda line : line.split(","))

In [8]:
# Hacemos la union de los dos rdd
deportistaRDD = deportistaRDD.union(deportista2RDD)

In [9]:
#contamos la cantidad de rdds
deportistaRDD.count()

135572

In [ ]:
# ver los primeros elementos para ver el campo en comun entre equiposRDD y deportistaRDD
equiposOlimpicosRDD.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [ ]:
deportistaRDD.top(2)



[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

## Obtener equipo y deportista

In [10]:
# es id de equiposRDD y equipo_id de deportistsRDD lo que los une
# Se selecciona la última columna del RDD (equipo_id) que es valor eje y el resto de contenidos
 # Se selecciona solo el id que es el valor eje y la sigla del país


deportistasEquipos = deportistaRDD.map(lambda line : [line[-1],line[:-1]]).join(equiposOlimpicosRDD.map(lambda x : [x[0],x[2]]))#.takeSample(False,6,25)  #False si quiero que no se repitan , 6 cuantos quiero en la salida  y 25 la semilla de aleatoriedad

In [25]:
lista_deportisata_equipo = deportistaRDD.map(lambda line : [line[-1],line[:-1]]).join(equiposOlimpicosRDD.map(lambda x : [x[0],x[2]])).top(10)


In [31]:
lista_deportisata_equipo[:]

[('999', (['92679', 'Trygve Bjarne Pedersen', '1', '35', '0', '0'], 'NOR')),
 ('999', (['1144', 'Henrik Agersborg', '1', '47', '0', '0'], 'NOR')),
 ('999', (['10765', 'Einar Berntsen', '1', '28', '0', '0'], 'NOR')),
 ('998',
  (['111659', 'G. Bernard Bernie Skinner', '1', '34', '182', '82'], 'CAN')),
 ('996', (['116030', 'Edward Eddy Stutterheim', '1', '39', '0', '0'], 'NED')),
 ('995', (['71728', 'Gordon Frederick Love', '1', '26', '0', '0'], 'FRA')),
 ('995', (['101553', 'Louis Roche', '1', '0', '0', '0'], 'FRA')),
 ('994', (['87856', 'Leo Anthony O Connell', '1', '20', '0', '0'], 'USA')),
 ('994', (['55319', 'Johnson', '1', '0', '0', '0'], 'USA')),
 ('994', (['53520', 'Henry Wood Jameson', '1', '21', '0', '0'], 'USA'))]

In [27]:
lista_deportisata_equipo[1][0]

'999'

In [28]:
lista_deportisata_equipo[1][0][0]

'9'

In [29]:
deportistasEquipos.map(lambda x : (x[1][0][0],x[1][0][1:],x[1][1])).top(2)

[('99999', ['Alexander Grant Alick Rennie', '1', '32', '182', '71'], 'RSA'),
 ('99998', ['Robert John Bob Renney', '1', '21', '178', '90'], 'AUS')]

In [32]:
# obtenemos info de resultado
resultado = sc.textFile(path + 'resultados.csv').map(lambda line : line.split(","))

In [33]:
resultado.take(7)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['1', 'NA', '1', '39', '1'],
 ['2', 'NA', '2', '49', '2'],
 ['3', 'NA', '3', '7', '3'],
 ['4', 'Gold', '4', '2', '4'],
 ['5', 'NA', '5', '36', '5'],
 ['6', 'NA', '5', '36', '6']]

In [34]:
#Filtrar solo a los que ganaron medalla
resultado = resultado.filter(lambda l : 'NA' not in l[1])

In [35]:
resultado.take(3)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4'],
 ['38', 'Bronze', '15', '7', '19']]

In [39]:
deportistaRDD.top(2)



[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [40]:
equiposOlimpicosRDD.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [49]:
# Juntamos el RDD de deportistas ,equiposcon el de resultado

# 1er parte juntamos deportistas con y sus equipos
deportistas_Equipos = deportistaRDD.map(lambda line : [line[-1],line[:-1]]).join(equiposOlimpicosRDD.map(lambda x : [x[0],x[2]]))


In [50]:
deportistas_Equipos.takeSample(False,6,25)

[('308', (['117713', 'Ahmed Amin Tabouzada', '1', '0', '0', '0'], 'EGY')),
 ('96', (['95174', 'Dsir Piryns', '1', '0', '0', '0'], 'BEL')),
 ('1178', (['99228', 'Viktor Ravnik', '1', '22', '179', '80'], 'YUG')),
 ('888', (['52248', 'Vaipava Nevo Ioane', '1', '28', '152', '62'], 'SAM')),
 ('716', (['108017', 'Neville Ian Scott', '1', '21', '187', '74'], 'NZL')),
 ('1019', (['39331', 'Werner Geeser', '1', '23', '182', '68'], 'SUI'))]

In [80]:
deportistas_Equipos.map(lambda x  : (x[1][0][0],(x[1][0][1:],x[1][1]))).take(2)

[('1', (['A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('2', (['A Lamusi', '1', '23', '170', '60'], 'CHN'))]

In [78]:
# 2da parte Juntamos con los resultados obteniendo las medallas de cada deportista haciendo la union con el deportista_id
deportistasGanadores = deportistas_Equipos.map(lambda x  : (x[1][0][0],(x[1][0][1:],x[1][1]))).join(resultado.map(lambda y : (y[2],y[1])))

In [81]:
deportistasGanadores.take(2)

[('17996', ((['Cao Mianying', '2', '21', '176', '71'], 'CHN'), 'Silver')),
 ('18005', ((['Cao Yuan', '1', '17', '160', '42'], 'CHN'), 'Gold'))]

Obtenemos los puntajes de cada pais de acuerdo al valor de las medallas

In [41]:
# Diccionario valores de las medallas
valoresMedallas = {'Gold':7,'Silver':5,'Bronze':4}

In [82]:
# primer parte se reduce la tupla
# x[1][0][1] -> Iniciales pais
# x[1][1] -> tipo de medalla
deportistasGanadores.map(lambda x : (x[1][0][1],x[1][1])).take(2)

[('CHN', 'Silver'), ('CHN', 'Gold')]

In [83]:
# Segunda parte obtenemos los valores de cada medalla
deportistasGanadores.map(lambda x : (x[1][0][1],valoresMedallas[x[1][1]])).take(2)

[('CHN', 5), ('CHN', 7)]

In [84]:
# Guardamos en una  variable el rdd donde esta pais , puntos por medalla
paisesPuntos = deportistasGanadores.map(lambda x : (x[1][0][1],valoresMedallas[x[1][1]]))

In [85]:
paisesPuntos.take(2)

[('CHN', 5), ('CHN', 7)]

In [86]:
# hacemos la sumatoria por pais y ordenamos los resultados
from operator import add
conclusion = paisesPuntos.reduceByKey((add)).sortBy(lambda x : x[1],ascending = False)


In [87]:
conclusion.take(10)

[('USA', 32137),
 ('URS', 14834),
 ('GBR', 10925),
 ('GER', 10896),
 ('FRA', 9265),
 ('ITA', 8755),
 ('SWE', 8110),
 ('CAN', 7209),
 ('AUS', 6755),
 ('HUN', 6142)]

# Dataframes

In [88]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.storagelevel import StorageLevel
import pyspark.sql
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, IntegerType, StringType,FloatType
from pyspark.sql.types import Row

In [89]:
path = '/content/drive/MyDrive/Spark/files'

Creacion de schema

In [91]:
juegosSchema = StructType([StructField('juego_id',IntegerType(),False),# False si es obligatorio
                           StructField('nombre_juego',StringType(),False),
                           StructField("año",StringType(),False),
                           StructField("temporada",StringType(),False),
                           StructField("ciudad",StringType(),False)])

Lectura de schema

In [93]:
# Create a SparkSession
spark = SparkSession.builder.appName("DataFrameExample").getOrCreate()


In [94]:
# "header", "true" → Le dice a Spark que la primera fila del CSV contiene los nombres de las columnas.
juegoDF = spark.read.schema(juegosSchema).option ("header","true").csv(path + '/juegos.csv')

In [96]:
juegoDF.show(5)

+--------+------------+----+---------+---------+
|juego_id|nombre_juego| año|temporada|   ciudad|
+--------+------------+----+---------+---------+
|       1| 1896 Verano|1896|   Verano|   Athina|
|       2| 1900 Verano|1900|   Verano|    Paris|
|       3| 1904 Verano|1904|   Verano|St. Louis|
|       4| 1906 Verano|1906|   Verano|   Athina|
|       5| 1908 Verano|1908|   Verano|   London|
+--------+------------+----+---------+---------+
only showing top 5 rows



In [97]:
spark

In [99]:
spark.stop()

In [98]:
sc.stop()